# IMPORTA BIBLIOTECAS

In [1]:
import time
import json
import pandas as pd
from datetime import datetime

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException


from webdriver_manager.chrome import ChromeDriverManager
from dotenv import load_dotenv
import os

In [2]:
# Carregar credenciais do arquivo .env
load_dotenv()
LINKEDIN_EMAIL = os.getenv("LINKEDIN_EMAIL")
LINKEDIN_PASSWORD = os.getenv("LINKEDIN_PASSWORD")

# CONFIGURAÇÃO DO AMBIENTE

In [3]:
# Configuração do Selenium
def setup_driver():
    options = Options()
    options.add_argument("--start-maximized")
    options.add_argument("--disable-notifications")
    service = Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service, options=options)
    return driver

In [4]:
# Autenticar no LinkedIn
def linkedin_login(driver):
    driver.get("https://www.linkedin.com/login")
    time.sleep(2)

    email_field = driver.find_element(By.ID, "username")
    password_field = driver.find_element(By.ID, "password")

    email_field.send_keys(LINKEDIN_EMAIL)
    password_field.send_keys(LINKEDIN_PASSWORD)
    password_field.send_keys("\ue007")  # Pressionar Enter
    time.sleep(3)


In [5]:
# Adicionar rolagem na página
def scroll_down(driver):
    driver.execute_script("window.scrollBy(0, 300);")
    time.sleep(0.5)  # Esperar um pouco para que novos elementos sejam carregados

# PROCESSO DO SCRAPING

In [6]:
# Extrair conexões
def extract_connections(driver, profile_url):
    driver.get(profile_url)
    time.sleep(3)

    extracted_data = {}

    try:
        # Clicar no link de conexões
        WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable(
                (
                    By.CLASS_NAME,
                    "jBoWjCkDnQAudKFlfzuYKZdcceDKXE.inline-flex.align-items-center.link-without-hover-visited.pt2",
                )
            )
        ).click()
        time.sleep(2)

        # Inicializar a lista de nomes extraídos
        names = []

        while True:
            # Extração dos nomes na página atual
            name_elements = driver.find_elements(
                By.CLASS_NAME,
                "sVERDkcTlYPzPQlJOoKTEBiRQdqFMDVxijI.KSOPGGTUwpqguKseMcspzYpZImxdaDTs ",
            )
            # print([name.text.split('\n')[0] for name in name_elements if name.text])
            names.extend(
                [name.text.split("\n")[0] for name in name_elements if name.text]
            )

            # Tentar ir para a próxima página
            try:
                scroll_down(driver)
                next_button = driver.find_element(
                    By.XPATH, "//button[@aria-label='Avançar']"
                )
                if "disabled" in next_button.get_attribute("class"):
                    break  # Botão de próximo desabilitado, última página
                next_button.click()
                time.sleep(2)
            except NoSuchElementException:
                break  # Botão de próximo não encontrado, última página

        # Adicionar ao dicionário de dados extraídos
        extracted_data[profile_url] = names
        print(
            f"Conexões extraídas para {profile_url}: {len(names)} conexões encontradas."
        )

    except Exception as e:
        print(f"Erro ao acessar {profile_url}: {e}")

    return extracted_data

# CONSTRUÇÃO DO GRAFO

In [7]:
# Iterar pelas conexões e salvar resultados em JSON
def process_connections(file_path, output_path, driver):
    connections_df = pd.read_csv(file_path, sep=";")
    total = connections_df.shape[0]

    try:
        with open(output_path, "r", encoding="utf-8") as json_file:
            all_data = json.load(json_file)
    except (FileNotFoundError, json.JSONDecodeError):
        all_data = {}
    

    for index, row in connections_df.iterrows():
        profile_url = row["URL"]
        # Ignorar perfis já processados
        if profile_url in all_data:
            print(f"Perfil já processado: {row['First Name']} {row['Last Name']} => {profile_url}. Ignorando...")
            continue

        print(f"Processando {index + 1}/{total}: {row['First Name']} {row['Last Name']} => {profile_url} ==> {datetime.now()}")

        data = extract_connections(driver, profile_url)
        all_data.update(data)

        # Salvar os dados extraídos em um arquivo JSON
        with open(output_path, "w", encoding="utf-8") as json_file:
            json.dump(all_data, json_file, ensure_ascii=False, indent=4)

    print(f"Dados salvos em {output_path}")
    return all_data

# DEFINIÇÃO DO PIPELINE

In [8]:
# Pipeline principal
def run_pipeline(file_path, output_path):
    # Configurações de entrada e saída
    # file_path = "Connections.csv"  # Substitua pelo caminho correto
    # output_path = "extracted_connections.json"  # Nome do arquivo de saída

    print(f"Processo Iniciado ==> {datetime.now()}")
    print(50 * "==")

    # Configuração do Selenium
    driver = setup_driver()
    print(f"Driver Configurado ==> {datetime.now()}")
    print(50 * "==")

    # Login no LinkedIn
    linkedin_login(driver)
    print(f"Login Realizado ==> {datetime.now()}")
    print(50 * "==")

    # Processar conexões e salvar dados
    all_data = process_connections(file_path, output_path, driver)
    print(50 * "==")
    
    print(f"Finalizado Processamento ==> {datetime.now()}")
    # Fechar o navegador
    driver.quit()

    return all_data

# EXECUÇÃO DO PROCESSO

In [9]:
# Executar no Jupyter Notebook
file_path = (
    "..\small_world_linkedin\data\linkedin_urls.csv"  # Substitua pelo caminho correto
)
output_path = (
    "..\small_world_linkedin\data\linkedin_graph.json"  # Nome do arquivo de saída
)

In [ ]:
all_data = run_pipeline(file_path, output_path)